The basic steps we'll take are:

1. Use DuckDuckGo to search for images of "dogs photos".
2. Fine-tune a pretrained neural network to recognise these two groups
3. Try running this model on a picture of a bird and see if it works.

In [ ]:
%pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
# help(fastbook)

In [ ]:
from fastbook import *

In [ ]:
# !pip install -Uqq duckduckgo_search

## Step 1: Download images of dogs

In [ ]:
# from duckduckgo_search import DDGS
# from fastcore.all import *

# ddgs = DDGS()

# def search_images(term, max_images=30):
#     print(f"Searching for '{term}'")
#     return L(ddgs.images(term, max_results=max_images)).itemgot('image')

In [ ]:
from fastbook import search_images_ddg

results = search_images_ddg('german shepherd dogs', max_images=1)
images = results.attrgot('content_url')
len(images)
results

In [ ]:
from fastdownload import download_url
dest = 'images/dogs.jpg'
download_url(results[0], dest, show_progress=False)

In [ ]:
dog_types = ['german shepherd', 'black', 'labrador']
path = Path('dogs')

In [ ]:
if not path.exists():
    path.mkdir()
    for t in dog_types:
        dest = (path/t)
        print(dest)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(f'{t} dog')
        download_images(dest, urls=results)

In [ ]:
files = get_image_files(path)
files

In [ ]:
corrupt = verify_images(files)
corrupt

In [ ]:
corrupt.map(Path.unlink);

## Step 2: Train our model

Para convertir nuestros datos descargados en un objeto DataLoaders, debemos decirle a fastai al menos cuatro cosas:

- ¿Con qué tipo de datos estamos trabajando?
- Cómo obtener la lista de artículos
- Cómo etiquetar estos artículos
- Cómo crear el conjunto de validación

In [ ]:
dogs = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = dogs.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

### Resize

De forma predeterminada, Cambiar tamaño recorta las imágenes para que se ajusten a una forma cuadrada del tamaño solicitado, utilizando todo el ancho o alto. Esto puede provocar la pérdida de algunos detalles importantes. Alternativamente, puedes pedirle a fastai que rellene las imágenes con ceros (negro), o que las aplaste/estire:

In [ ]:
dogs = dogs.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = dogs.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

Aquí hay otro ejemplo en el que reemplazamos Resize con RandomResizedCrop, que es la transformación que proporciona el comportamiento que acabamos de describir. El parámetro más importante a pasar es min_scale, que determina qué parte de la imagen seleccionar como mínimo cada vez:

In [ ]:
dogs = dogs.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = dogs.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

### Data Augmentation

El aumento de datos se refiere a la creación de variaciones aleatorias de nuestros datos de entrada, de modo que parezcan diferentes, pero en realidad no cambien el significado de los datos.

In [ ]:
dogs = dogs.new(
        item_tfms=RandomResizedCrop(224, min_scale=0.5),
        batch_tfms=aug_transforms(mult=2)
        )
dls = dogs.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

## Step 3: Training Your Model, and Using It to Clean Your Data

Ahora veamos si los errores que comete el modelo son principalmente pensar que los osos pardos son ositos de peluche (¡eso sería malo para la seguridad!), o que los osos pardos son osos negros, o algo más. Para visualizar esto, podemos crear una matriz de confusión:

In [ ]:
dogs = dogs.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = dogs.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

Ahora veamos si los errores que comete el modelo son principalmente pensar que los pastores alemanes son labradores, o que los labradores son perros negros, o algo más. Para visualizar esto, podemos crear una matriz de confusión:

In [ ]:
from fastai.vision.widgets import *
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()


Las filas representan todos los perros black, german y labrador de nuestro conjunto de datos, respectivamente. Las columnas representan las imágenes que el modelo predijo como perros black, german y labrador, respectivamente. Por lo tanto, la diagonal de la matriz muestra las imágenes que se clasificaron correctamente y las celdas fuera de la diagonal representan aquellas que se clasificaron incorrectamente.

Es útil ver dónde ocurren exactamente nuestros errores, ver si se deben a un problema del conjunto de datos (por ejemplo, imágenes que no son perros en absoluto o están etiquetadas incorrectamente, etc.) o a un problema de modelo (quizás no maneja imágenes tomadas con iluminación inusual, o desde un ángulo diferente, etc.). Para ello, podemos ordenar nuestras imágenes por su pérdida.

La pérdida es un número que es mayor si el modelo es incorrecto (especialmente si también confía en su respuesta incorrecta), o si es correcto, pero no confía en su respuesta correcta. En un par de capítulos aprenderemos en profundidad cómo se calcula y utiliza la pérdida en el proceso de capacitación. Por ahora, plot_top_losses nos muestra las imágenes con mayor pérdida en nuestro conjunto de datos. Como dice el título del resultado, cada imagen está etiquetada con cuatro cosas: predicción, real (etiqueta de destino), pérdida y probabilidad. La probabilidad aquí es el nivel de confianza, de cero a uno, que el modelo ha asignado a su predicción:

In [ ]:
interp.plot_top_losses(6, nrows=3)

Este resultado muestra que la imagen con mayor pérdida es aquella que se ha predicho como "labrador" con alta confianza. Sin embargo, está etiquetado (según nuestra búsqueda de imágenes en Bing) como "negro". No somos expertos en perros, ¡pero seguro que nos parece que esta etiqueta es incorrecta! Probablemente deberíamos cambiar su etiqueta a "labrardor".

El enfoque intuitivo para realizar la limpieza de datos es hacerlo antes de entrenar un modelo. Pero, como ha visto en este caso, un modelo puede ayudarle a encontrar problemas de datos de forma más rápida y sencilla. Por lo tanto, normalmente preferimos entrenar primero un modelo rápido y simple y luego usarlo para ayudarnos con la limpieza de datos.

Cada imagen está etiquetada con cuatro cosas: predicción, real (target label), pérdida y probabilidad.
La probabilidad aquí es el nivel de confianza, de cero a uno, que el modelo ha asignado a su predicción

### Data cleaning

fastai incluye una práctica GUI para la limpieza de datos llamada ImageClassifierCleaner que le permite elegir una categoría y el conjunto de entrenamiento versus validación y ver las imágenes con mayor pérdida (en orden), junto con menús que permiten seleccionar imágenes para eliminarlas o volver a etiquetarlas:

In [ ]:
#hide_output
cleaner = ImageClassifierCleaner(learn)
cleaner

Entonces, por ejemplo, para eliminar (desvincular) todas las imágenes seleccionadas para su eliminación, ejecutaríamos:

- for idx in cleaner.delete(): cleaner.fns[idx].unlink()

Para mover imágenes para las que hemos seleccionado una categoría diferente, ejecutaríamos:

- for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
#hide
# for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

Una vez que hayamos limpiado nuestros datos, podemos volver a entrenar nuestro modelo.

## Step 4: How to Export the Trained Model

In [ ]:
learn.export()
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
model_inf = load_learner(path/'export.pkl')

In [ ]:
model_inf.predict('/content/dogs/black/044331f5-3c7d-4cd0-908c-b1ee4911b9c9.jpg')

In [ ]:
model_inf.dls.vocab